In [24]:
batch = [
    {'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': 'Bonjour'},
    {'instruction': 'Sum', 'input': '1 2', 'output': '3'},
    {'instruction': 'Capitalize', 'input': 'world', 'output': 'our real world'},
]

from transformers import AutoTokenizer  # 从transformers库导入AutoTokenizer类

model_path = "../results/checkpoint-103520"
# 加载微调后模型 (SFT) 和 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)

In [25]:
instructions = [sample['instruction'] for sample in batch]
inputs = [sample['input'] for sample in batch]
outputs = [sample['output'] for sample in batch]
input_texts = [f"{instruction} {input} {output}" for instruction, input, output in zip(instructions, inputs, outputs)] 

In [26]:
tokenized_inputs = tokenizer(
    input_texts,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=tokenizer.model_max_length,
    add_special_tokens=True
)

labels = tokenized_inputs["input_ids"].clone()
for i, feature in enumerate(batch):
    instruction_length = len(tokenizer(feature['instruction'], add_special_tokens=False)["input_ids"])
    if feature['input'] is None:
        feature['input'] = ""
    input_length = len(tokenizer(feature['input'], add_special_tokens=False)["input_ids"])
    output_start = instruction_length + input_length+1
    print(output_start)
    labels[i, :output_start] = -100

8
5
4


In [27]:
for i, (input_id, label) in enumerate(zip(tokenized_inputs["input_ids"], labels)):
    print(f"Sample {i+1}:")
    print(f"Input IDs: {input_id}")
    print(f"Labels: {label}")
    print("-" * 40)

Sample 1:
Input IDs: tensor([35127,  2326, 10414,   369, 19429,  9314,    13,   220, 13481, 29262])
Labels: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 13481, 29262])
----------------------------------------
Sample 2:
Input IDs: tensor([  9190,    220,     16,    220,     17,    220,     18, 151643, 151643,
        151643])
Labels: tensor([  -100,   -100,   -100,   -100,   -100,    220,     18, 151643, 151643,
        151643])
----------------------------------------
Sample 3:
Input IDs: tensor([ 12903,  34313,   1879,   1039,   1931,   1879, 151643, 151643, 151643,
        151643])
Labels: tensor([  -100,   -100,   -100,   -100,   1931,   1879, 151643, 151643, 151643,
        151643])
----------------------------------------


In [28]:
instruction_inputs = [f"{instruction} {input}" for instruction, input in zip(instructions, inputs)]


In [29]:
tokenized_instruction_inputs = tokenizer(
    instruction_inputs,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=tokenizer.model_max_length
)

for i in range(len(batch)):
    print(f"Sample {i+1}:")
    print(f"Full Input IDs: {tokenized_inputs['input_ids'][i]}")
    print(f"Instruction+Input IDs: {tokenized_instruction_inputs['input_ids'][i]}")
    print("-" * 40)

Sample 1:
Full Input IDs: tensor([35127,  2326, 10414,   369, 19429,  9314,    13,   220, 13481, 29262])
Instruction+Input IDs: tensor([35127,  2326, 10414,   369, 19429,  9314,    13,   220])
----------------------------------------
Sample 2:
Full Input IDs: tensor([  9190,    220,     16,    220,     17,    220,     18, 151643, 151643,
        151643])
Instruction+Input IDs: tensor([  9190,    220,     16,    220,     17, 151643, 151643, 151643])
----------------------------------------
Sample 3:
Full Input IDs: tensor([ 12903,  34313,   1879,   1039,   1931,   1879, 151643, 151643, 151643,
        151643])
Instruction+Input IDs: tensor([ 12903,  34313,   1879, 151643, 151643, 151643, 151643, 151643])
----------------------------------------


#### 原模型生成能力

In [23]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 确保CUDA可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载原始预训练模型 (PLM) 和 tokenizer
plm_tokenizer = AutoTokenizer.from_pretrained("qwen2.5", use_fast=False, trust_remote_code=True)
plm_model = AutoModelForCausalLM.from_pretrained(
    "qwen2.5", 
    device_map=None, 
    torch_dtype=torch.bfloat16
).to(device)

# 加载微调后模型 (SFT) 和 tokenizer
sft_tokenizer = AutoTokenizer.from_pretrained("results\checkpoint-103520", use_fast=False, trust_remote_code=True)
sft_model = AutoModelForCausalLM.from_pretrained(
    "results\checkpoint-103520", 
    device_map=None, 
    torch_dtype=torch.bfloat16
).to(device)

# 输入指令
input_text = "My cell phone fell into the water. What should I do?"

# 定义生成函数
def generate_response(model, tokenizer, input_text, max_length=10):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        do_sample=True,
        num_return_sequences=1
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# # 生成预训练模型 (PLM) 的结果
# plm_response = generate_response(plm_model, plm_tokenizer, input_text)
# print("预训练模型 (PLM) 生成结果：")
# print(plm_response)

# 生成微调后模型 (SFT) 的结果
sft_response = generate_response(sft_model, sft_tokenizer, input_text)
print("微调后模型 (SFT) 生成结果：")
print(sft_response)


<>:16: SyntaxWarning: invalid escape sequence '\c'
<>:18: SyntaxWarning: invalid escape sequence '\c'
<>:16: SyntaxWarning: invalid escape sequence '\c'
<>:18: SyntaxWarning: invalid escape sequence '\c'
C:\Windows\Temp\ipykernel_33316\924351632.py:16: SyntaxWarning: invalid escape sequence '\c'
  sft_tokenizer = AutoTokenizer.from_pretrained("results\checkpoint-103520", use_fast=False, trust_remote_code=True)
C:\Windows\Temp\ipykernel_33316\924351632.py:18: SyntaxWarning: invalid escape sequence '\c'
  "results\checkpoint-103520",
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/

微调后模型 (SFT) 生成结果：
My cell phone fell into the water. What should I do? None If your cell phone falls into water, the most important thing is to get it out of the water as soon as possible. Here are some steps you can follow:

1. If you have a phone that has a built-in water-resistant battery, try to use it to call for help or send an emergency text message.

2. If you have a phone with a removable battery, you can try to remove the battery and plug it into a power source, such as a laptop or a charger.

3. If you are using a mobile phone, try to hold the phone above the water, away from your face, to prevent water from entering your ear. If you are using a smartphone with a water-resistant screen, it may still be able to protect your phone from the water.

4. If you are unable to get your phone out of water, immediately contact a professional for help.

It is important to remember that water can damage phones and other electronic devices, so it is always a good idea to keep them in a d